In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
sys.path.append(".")
import torch
import numpy as np
import copy
import plotly
import plotly.graph_objs as go
import pickle
from datetime import datetime
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.samples_generator as samples_generator
import robust_value_approx.samples_buffer as samples_buffer
import robust_value_approx.value_approximation as value_approximation
import robust_value_approx.training_log as training_log
import robust_value_approx.controllers as controllers

import pendulum_utils
import acrobot_utils

/home/blandry/Code/robust-value-approx/robust_value_approx/value_approximation.py:2: _DrakeImportWarning:


You may have already (directly or indirectly) imported `torch` which uses
`RTLD_GLOBAL`. Using `RTLD_GLOBAL` may cause symbol collisions which manifest
themselves in bugs like "free(): invalid pointer". Please consider importing
`pydrake` (and related C++-wrapped libraries like `cv2`, `open3d`, etc.)
*before* importing `torch`. For more details, see:
https://github.com/pytorch/pytorch/issues/3059#issuecomment-534676459




# Pendulum

In [343]:
pendulum = pendulum_utils.PendulumNLP()
vf = pendulum_utils.get_value_function(5)
V = vf.get_value_function()

In [344]:
x0 = torch.Tensor([np.pi+1.5, 15.]).double()
x_traj, u_traj, alpha_traj = vf.sol_to_traj(x0, *(V(x0)[1:]))
fig = go.Figure()
for i in range(2):
    fig.add_trace(go.Scatter(
        y=x_traj[i,:]
    ))
fig.show()

In [172]:
# ctrl = lambda x: (torch.zeros(1), torch.zeros(1), None)
ctrl = controllers.get_optimal_controller(vf)
# u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
# u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
# ctrl = controllers.get_lqr_controller(pendulum.dx, x0, torch.zeros(u_min.shape[0]).type(vf.dtype), vf.Q[0], vf.R[0], u_min, u_max)

x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, 1, pendulum.dx, ctrl, .1, 50)

fig = go.Figure()
for i in range(2):
    fig.add_trace(go.Scatter(
        x=t_traj_sim,
        y=x_traj_sim[i,:]
    ))
fig.show()

In [373]:
x0_lo = torch.Tensor([np.pi-1.5, -15.]).type(vf.dtype)
x0_up = torch.Tensor([np.pi+1.5, 15.]).type(vf.dtype)

# file options
sys_name = 'pendulum'
x_samples_file = '../data/validation_' + sys_name + '_x.pt'
v_samples_file = '../data/validation_' + sys_name + '_v.pt'
model_file = '../data/' + sys_name + '.pt'

In [419]:
opt = dict(
    max_buffer_size = 10,
    batch_size = 5,
    nn_width = 16,
    nn_depth = 0,

    init_num_samples = 0,
    init_num_trainig_step = 0,
    
    num_generations = 20,
    num_samples_per_generation = 10,
    num_train_step_per_gen = 1000,
    
    adv_max_iter = 2,
    adv_conv_tol = 1e-5,
    adv_learning_rate = 1.,
    
    num_samples_validation = 500,
)

In [420]:
samples_buff = samples_buffer.SamplesBuffer(x0_lo.shape[0]*(vf.N-1), vf.N-1, vf.dtype, max_size=opt['max_buffer_size'])
samples_gen = samples_generator.RandomSampleGenerator(vf, x0_lo, x0_up)
vf_approx = value_approximation.InfiniteHorizonValueFunctionApproximation(x0_lo, x0_up, opt['nn_width'], opt['nn_depth'])
train_log = training_log.TrainingLog(1, prefix="baseline")

In [391]:
x_samples_validation, v_labels_validation = samples_gen.generate_samples(opt['num_samples_validation'])

In [177]:
torch.save(x_samples_validation, x_samples_file)
torch.save(v_labels_validation, v_samples_file)

In [412]:
x_samples_validation = torch.load(x_samples_file)
v_labels_validation = torch.load(v_samples_file)

In [421]:
(x_samples, v_labels) = samples_gen.generate_samples(opt['init_num_samples'])
samples_buff.add_samples(x_samples, v_labels)

In [422]:
for train_step_i in range(opt['init_num_trainig_step']):
    x, v = samples_buff.get_random_samples(opt['batch_size'])
    losses = vf_approx.train_step(x, v)
    train_log.add_train_loss(losses)

In [423]:
samples_buff_adv = copy.deepcopy(samples_buff)
samples_gen_adv = samples_generator.AdversarialSampleGenerator(vf, x0_lo, x0_up, 
                                                               max_iter=opt['adv_max_iter'],
                                                               conv_tol=opt['adv_conv_tol'],
                                                               learning_rate=opt['adv_learning_rate'])
vf_approx_adv = copy.deepcopy(vf_approx)
train_log_adv = training_log.TrainingLog.get_copy(
    train_log, prefix="adversarial", keep_writer=True)

In [428]:
validation_losses = vf_approx_adv.validation_loss(x_samples_validation, v_labels_validation)
train_log_adv.add_validation_loss(validation_losses)
validation_losses = vf_approx.validation_loss(x_samples_validation, v_labels_validation)
train_log.add_validation_loss(validation_losses)
for gen_i in range(opt['num_generations']):
    # adverserial samples
    (x_samples, v_labels) = samples_gen_adv.generate_samples(opt['num_samples_per_generation'], vf_approx_adv)
    samples_buff_adv.add_samples(x_samples, v_labels)
    # random samples
    (x_samples, v_labels) = samples_gen.generate_samples(opt['num_samples_per_generation'])
    samples_buff.add_samples(x_samples, v_labels)
    for train_step_i in range(opt['num_train_step_per_gen']):
        samples_indices = samples_buff_adv.get_random_sample_indices(opt['batch_size'])
        x, v = samples_buff_adv.get_samples_from_indices(samples_indices)
        losses = vf_approx_adv.train_step(x, v)
        train_log_adv.add_train_loss(losses)
        x, v = samples_buff.get_samples_from_indices(samples_indices)
        losses = vf_approx.train_step(x, v)
        train_log.add_train_loss(losses)
    validation_losses = vf_approx_adv.validation_loss(x_samples_validation, v_labels_validation)
    train_log_adv.add_validation_loss(validation_losses)
    validation_losses = vf_approx.validation_loss(x_samples_validation, v_labels_validation)
    train_log.add_validation_loss(validation_losses)

In [429]:
dx = pendulum.dx
step_cost = vf.step_cost
u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
dt_ctrl = .2
num_samples = 50

ctrl = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx.model, u_min, u_max, dt_ctrl, num_samples)
x0 = torch.Tensor([np.pi+0.5, 1.]).double()
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, 1, dx, ctrl, .1, 100)

fig = go.Figure()
for i in range(2):
    fig.add_trace(go.Scatter(
        x=t_traj_sim,
        y=x_traj_sim[i,:]
    ))
fig.show()

In [430]:
# controller params
dx = pendulum.dx
step_cost = vf.step_cost
u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
dt_ctrl = .2
num_samples = 40

# benchmark params
x0 = torch.Tensor([np.pi, 0.]).type(vf.dtype)
# x0_eps = torch.Tensor([1.5, 15.]).type(vf.dtype)
x0_eps = torch.Tensor([2., 30.]).type(vf.dtype)


x_goal = torch.Tensor([np.pi, 0.]).type(vf.dtype)
num_breaks = [10, 10]
dt_sim = .1
N = 10

In [431]:
ctrl_adv = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx_adv.model, u_min, u_max, dt_ctrl, num_samples)
bench_adv = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_adv, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [432]:
ctrl_baseline = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx.model, u_min, u_max, dt_ctrl, num_samples)
bench_baseline = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_baseline, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [433]:
ctrl_lqr = controllers.get_lqr_controller(pendulum.dx, x0, torch.zeros(u_min.shape[0]).type(vf.dtype), vf.Q[0], vf.R[0], u_min, u_max)
bench_lqr = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_lqr, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [434]:
conv_thresh = .8
zmin = 0.
zmax = 3.

baseline_plot = True
adversarial_plot = True
lqr_plot = True

if baseline_plot:
    print("Baseline")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = bench_baseline.detach().numpy(),
        zmin = zmin,
        zmax = zmax,
        ))
    fig.show()

if adversarial_plot:
    print("Adversarial")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = bench_adv.detach().numpy(),
        zmin = zmin,
        zmax = zmax,
        ))
    fig.show()

if lqr_plot:
    print("LQR")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = bench_lqr.detach().numpy(),
        zmin = zmin,
        zmax = zmax,
        ))
    fig.show()

if baseline_plot:
    print("Baseline")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = torch.Tensor(bench_baseline.detach().numpy() > conv_thresh).type(vf.dtype),
        ))
    fig.show()

if adversarial_plot:
    print("Adversarial")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = torch.Tensor(bench_adv.detach().numpy() > conv_thresh).type(vf.dtype),
        ))
    fig.show()

if lqr_plot:
    print("LQR")
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = torch.Tensor(bench_lqr.detach().numpy() > conv_thresh).type(vf.dtype),
        ))
    fig.show()

Baseline


Adversarial


LQR


Baseline


Adversarial


LQR


In [ ]:
if baseline_plot:
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z = (bench_baseline-bench_adv).detach().numpy(),
        ))
    fig.show()

# Acrobot

In [320]:
acrobot = acrobot_utils.AcrobotNLP()
vf = acrobot_utils.get_value_function(10)
V = vf.get_value_function()

In [321]:
eps = .1
x0 = torch.Tensor([np.pi+eps, -eps, 0., 0.]).double()
x_traj, u_traj, alpha_traj = vf.sol_to_traj(x0, *(V(x0)[1:]))
fig = go.Figure()
for i in range(4):
    fig.add_trace(go.Scatter(
        y=x_traj[i,:]
    ))
fig.show()

In [325]:
# ctrl = lambda x: (torch.zeros(1), torch.zeros(1), None)

ctrl = controllers.get_optimal_controller(vf)

x0 = torch.Tensor([np.pi, 0., 0., 0.]).double()
u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
Q = np.diag([1., 1., .1, .1])
R = np.diag([.1])
# ctrl = controllers.get_lqr_controller(acrobot.dx, x0, torch.zeros(u_min.shape[0]).type(vf.dtype), Q, R, u_min, u_max)

eps = 0.1
x0 = torch.Tensor([np.pi+eps, -eps, 0., 0.]).double()
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, 1, acrobot.dx, ctrl, .1, 100)

fig = go.Figure()
for i in range(4):
    fig.add_trace(go.Scatter(
        x=t_traj_sim,
        y=x_traj_sim[i,:]
    ))
fig.show()

In [326]:
eps = .1
x0_lo = torch.Tensor([np.pi-eps, -eps, -5., -5.]).type(vf.dtype)
x0_up = torch.Tensor([np.pi+eps, eps, 5., 5.]).type(vf.dtype)

# file options
sys_name = 'acrobot'
x_samples_file = '../data/validation_' + sys_name + '_x.pt'
v_samples_file = '../data/validation_' + sys_name + '_v.pt'
model_file = '../data/' + sys_name + '.pt'

In [327]:
# opt = dict(
#     max_buffer_size = None,
#     batch_size = 100,
#     nn_width = 500,
#     nn_depth = 2,

#     init_num_samples = 10,
#     init_num_trainig_step = 100,
    
#     num_generations = 10,
#     num_samples_per_generation = 5,
#     num_train_step_per_gen = 100,
    
#     adv_max_iter = 5,
#     adv_conv_tol = 1e-5,
#     adv_learning_rate = 1.,
    
#     num_samples_validation = 250,
# )

opt = dict(
    max_buffer_size = 10,
    batch_size = 5,
    nn_width = 64,
    nn_depth = 0,

    init_num_samples = 5,
    init_num_trainig_step = 100,
    
    num_generations = 100,
    num_samples_per_generation = 10,
    num_train_step_per_gen = 100,
    
    adv_max_iter = 5,
    adv_conv_tol = 1e-5,
    adv_learning_rate = .25,
    
    num_samples_validation = 500,
)

In [328]:
samples_buff = samples_buffer.SamplesBuffer(x0_lo.shape[0]*(vf.N-1), vf.N-1, vf.dtype, max_size=opt['max_buffer_size'])
samples_gen = samples_generator.RandomSampleGenerator(vf, x0_lo, x0_up)

In [329]:
vf_approx = value_approximation.InfiniteHorizonValueFunctionApproximation(x0_lo, x0_up, opt['nn_width'], opt['nn_depth'])
train_log = training_log.TrainingLog(1, prefix="baseline")

In [102]:
x_samples_validation, v_labels_validation = samples_gen.generate_samples(opt['num_samples_validation'])

In [103]:
torch.save(x_samples_validation, x_samples_file)
torch.save(v_labels_validation, v_samples_file)

In [330]:
x_samples_validation = torch.load(x_samples_file)
v_labels_validation = torch.load(v_samples_file)

In [331]:
(x_samples, v_labels) = samples_gen.generate_samples(opt['init_num_samples'])
samples_buff.add_samples(x_samples, v_labels)

In [332]:
for train_step_i in range(opt['init_num_trainig_step']):
    x, v = samples_buff.get_random_samples(opt['batch_size'])
    losses = vf_approx.train_step(x, v)
    train_log.add_train_loss(losses)

In [333]:
samples_buff_adv = copy.deepcopy(samples_buff)
samples_gen_adv = samples_generator.AdversarialSampleGenerator(vf, x0_lo, x0_up, 
                                                               max_iter=opt['adv_max_iter'],
                                                               conv_tol=opt['adv_conv_tol'],
                                                               learning_rate=opt['adv_learning_rate'])
vf_approx_adv = copy.deepcopy(vf_approx)
train_log_adv = training_log.TrainingLog.get_copy(
    train_log, prefix="adversarial", keep_writer=True)

In [334]:
validation_losses = vf_approx_adv.validation_loss(x_samples_validation, v_labels_validation)
train_log_adv.add_validation_loss(validation_losses)
validation_losses = vf_approx.validation_loss(x_samples_validation, v_labels_validation)
train_log.add_validation_loss(validation_losses)
for gen_i in range(opt['num_generations']):
    # adverserial samples
    (x_samples, v_labels) = samples_gen_adv.generate_samples(opt['num_samples_per_generation'], vf_approx_adv)
    samples_buff_adv.add_samples(x_samples, v_labels)
    # random samples
    (x_samples, v_labels) = samples_gen.generate_samples(opt['num_samples_per_generation'])
    samples_buff.add_samples(x_samples, v_labels)
    for train_step_i in range(opt['num_train_step_per_gen']):
        samples_indices = samples_buff_adv.get_random_sample_indices(opt['batch_size'])
        x, v = samples_buff_adv.get_samples_from_indices(samples_indices)
        losses = vf_approx_adv.train_step(x, v)
        train_log_adv.add_train_loss(losses)
        x, v = samples_buff.get_samples_from_indices(samples_indices)
        losses = vf_approx.train_step(x, v)
        train_log.add_train_loss(losses)
    validation_losses = vf_approx_adv.validation_loss(x_samples_validation, v_labels_validation)
    train_log_adv.add_validation_loss(validation_losses)
    validation_losses = vf_approx.validation_loss(x_samples_validation, v_labels_validation)
    train_log.add_validation_loss(validation_losses)

KeyboardInterrupt: 

In [130]:
dx = acrobot.dx
step_cost = vf.step_cost
u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
dt_ctrl = .2
num_samples = 100

ctrl = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx.model, u_min, u_max, dt_ctrl, num_samples)
eps = 0.
x0 = torch.Tensor([np.pi+eps, -eps, 0., 0.]).double()
x_traj_sim, t_traj_sim = controllers.sim_ctrl(x0, 1, acrobot.dx, ctrl, .1, 50)

fig = go.Figure()
for i in range(2):
    fig.add_trace(go.Scatter(
        x=t_traj_sim,
        y=x_traj_sim[i,:]
    ))
fig.show()

In [ ]:
# controller params
dx = acrobot.dx
step_cost = vf.step_cost
u_min = torch.Tensor(vf.u_lo[0]).type(vf.dtype)
u_max = torch.Tensor(vf.u_up[0]).type(vf.dtype)
dt_ctrl = .1
num_samples = 10

# benchmark params
x0 = torch.Tensor([np.pi, 0., 0., 0.]).type(vf.dtype)
x0_eps = torch.Tensor([.1, .1, 0., 0.]).type(vf.dtype)
x_goal = torch.Tensor([np.pi, 0., 0., 0.]).type(vf.dtype)
num_breaks = [5, 5]
dt_sim = .1
N = 20

In [ ]:
ctrl_adv = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx_adv.model, u_min, u_max, dt_ctrl, num_samples)
bench_adv = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_adv, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [ ]:
ctrl_baseline = controllers.get_sampling_infinite_horizon_controller(dx, step_cost, vf_approx.model, u_min, u_max, dt_ctrl, num_samples)
bench_baseline = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_baseline, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [ ]:
ctrl_lqr = controllers.get_lqr_controller(pendulum.dx, x0, torch.zeros(u_min.shape[0]).type(vf.dtype), vf.Q[0], vf.R[0], u_min, u_max)
bench_lqr = controllers.benchmark_controller(u_min.shape[0], dx, ctrl_lqr, x0, x0_eps, num_breaks, x_goal, dt_sim, N)

In [ ]:
conv_thresh = 1.
zmin = 0.
zmax = 5.

print("Baseline")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = bench_baseline.detach().numpy(),
    zmin = zmin,
    zmax = zmax,
    ))
fig.show()

print("Adversarial")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = bench_adv.detach().numpy(),
    zmin = zmin,
    zmax = zmax,
    ))
fig.show()

print("LQR")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = bench_lqr.detach().numpy(),
    zmin = zmin,
    zmax = zmax,
    ))
fig.show()

print("Baseline")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = torch.Tensor(bench_baseline.detach().numpy() > conv_thresh).type(vf.dtype),
    ))
fig.show()

print("Adversarial")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = torch.Tensor(bench_adv.detach().numpy() > conv_thresh).type(vf.dtype),
    ))
fig.show()

print("LQR")
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z = torch.Tensor(bench_lqr.detach().numpy() > conv_thresh).type(vf.dtype),
    ))
fig.show()